In [2]:
!pip install pandas

  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.4 MB 8.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.4 MB 8.4 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.4 MB 11.4 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.4 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 11.4/11.4 MB 12.5 MB/s eta 0:00:00
Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   

In [4]:
import requests
import pandas as pd

API_KEY = "67ffece4b2ae08.94077168"
EXCHANGE = "US"
url = f"https://eodhd.com/api/exchange-symbol-list/{EXCHANGE}?api_token={API_KEY}&fmt=json"

try:
    response = requests.get(url)
    response.raise_for_status()
    ticker_data = response.json()
    
    # Load entire dataset as-is
    full_df = pd.DataFrame(ticker_data)
    print(f"Loaded {len(full_df)} tickers.")
    display(full_df.head(10))  # Show preview

except Exception as e:
    print(f"Error fetching or processing ticker data: {e}")


Loaded 51382 tickers.


,Code,Name,Country,Exchange,Currency,Type,Isin
0,0P000070L2,RBC $U.S. Money Market Fund A,USA,OTC,USD,FUND,None
1,0P0000A2WI,Fidelity American High Yield Sr B,USA,US,USD,FUND,None
2,0P0000A412,Franklin U.S. Opportunities Fund A,USA,PINK,USD,FUND,None
3,0P0000O6WE,Nh Unique College Investing Plan Fidelity 500 ...,USA,NMFQS,USD,FUND,None
4,0P0000O753,Oh Collegeadvantage 529 Vanguard Wellington Op...,USA,NMFQS,USD,FUND,None
5,0P0000RX5G,Franklin Floating Rate Fund PLC W USD Dis,USA,PINK,USD,FUND,None
6,0P0000V6X4,Vanguard Target Retire Trust Plus 2040,USA,US,USD,FUND,None
7,0P0000V6X7,Target Retirement 2055 Trust Plus,USA,NMFQS,USD,FUND,US92202V3446
8,0P00015SK3,Mfs Large Cap Value Cit Cl 3,USA,NMFQS,USD,FUND,None
9,0P0001GXZ7,Oaktree (Lux.) III - Oaktree Global Credit Fun...,USA,OTC,USD,FUND,LU1876555878


In [8]:
expected_tickers = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'AMD', 'GM', 'RDDT', 'U', 'SPXS']

# Check which expected tickers are present
found = full_df[full_df['Code'].isin(expected_tickers)]

# Check which ones are missing
missing = list(set(expected_tickers) - set(found['Code']))

print("✅ Found:")
display(found)

print("❌ Missing:")
print(missing)


✅ Found:


,Code,Name,Country,Exchange,Currency,Type,Isin
227,AAPL,Apple Inc,USA,NASDAQ,USD,Common Stock,US0378331005
1921,AMD,Advanced Micro Devices Inc,USA,NASDAQ,USD,Common Stock,US0079031078
19652,GM,General Motors Company,USA,NYSE,USD,Common Stock,US37045V1008
19956,GOOG,Alphabet Inc Class C,USA,NASDAQ,USD,Common Stock,US02079K1079
31274,MSFT,Microsoft Corporation,USA,NASDAQ,USD,Common Stock,US5949181045
38881,RDDT,"Reddit, Inc.",USA,NYSE,USD,Common Stock,US75734B1008
43107,SPXS,Direxion Daily S&P 500® Bear 3X Shares,USA,NYSE ARCA,USD,ETF,US25460E2651
46421,TSLA,Tesla Inc,USA,NASDAQ,USD,Common Stock,US88160R1014
46860,U,Unity Software Inc,USA,NYSE,USD,Common Stock,US91332U1016


❌ Missing:
[]


In [17]:
import requests
import pandas as pd

API_KEY = "67ffece4b2ae08.94077168"
base_url = "https://eodhd.com/api/screener"
headers = {"Content-Type": "application/json"}

tickers = []
limit = 500
total_desired = 5000

for offset in range(0, total_desired, limit):
    params = {
        "api_token": API_KEY,
        "sort": "avgvol_200d.desc",  # ✅ valid field
        "limit": limit,
        "offset": offset,
        "filters": '[["exchange","=","US"]]',
    }

    response = requests.get(base_url, params=params, headers=headers)
    data = response.json()

    if "data" not in data:
        print("❌ Error response:")
        print(data)
        break

    tickers += [item["code"] for item in data["data"]]

# Create DataFrame
top_5000_df = pd.DataFrame(tickers, columns=["Ticker"])
print(f"✅ Loaded {len(top_5000_df)} tickers.")
display(top_5000_df.head())


❌ Error response:
{'errors': {'offset': ['The offset may not be greater than 999.']}}
✅ Loaded 1000 tickers.


,Ticker
0,HMBL
1,NVDA
2,RDAR
3,TSLL
4,SOXL


In [21]:
# Test tickers
test_tickers = ["AAPL", "TSLA", "META", "U", "RDDT", "SNOW", "PLTR", "BIRD", "XOM", "BRK.B", 'GM', 'F', 'AMZN', 'VOO', 'AMC', 'GME', 'SPY', 'SHOP', 'LUV', 'MU']

# Check for presence
top_set = set(top_5000_df['Ticker'].str.upper())
found = [t for t in test_tickers if t.upper() in top_set]
missing = [t for t in test_tickers if t.upper() not in top_set]

print("✅ Found in top 5000:", found)
print("❌ Missing from top 5000:", missing)


✅ Found in top 5000: ['AAPL', 'TSLA', 'META', 'U', 'RDDT', 'SNOW', 'PLTR', 'XOM', 'GM', 'F', 'AMZN', 'VOO', 'AMC', 'GME', 'SPY', 'LUV', 'MU']
❌ Missing from top 5000: ['BIRD', 'BRK.B', 'SHOP']
